In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
nltk.download('wordnet')
nltk.download('stopwords')
#sklearn.decomposition.LatentDirichletAllocation

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Bruno\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Bruno\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
data = pd.read_csv('data.csv')
data.fillna('', inplace=True)
data.head()

,contest,problem_name,problem_statement,problem_tags
0,325,A,You are given n rectangles. The corners of rec...,"implementation,*1500"
1,325,B,Daniel is organizing a football tournament. He...,"binarysearch,math,*1800"
2,325,C,Piegirl has found a monster and a book about m...,"dfsandsimilar,graphs,shortestpaths,*2600"
3,325,D,"In a far away land, there exists a planet shap...","dsu,*2900"
4,325,E,Piegirl found the red button. You have one las...,"combinatorics,dfsandsimilar,dsu,graphs,greedy,..."


In [3]:
print(f'quantidade de problemas: {data.shape[0]}')
print(f'quantidade de contests: {len(data.contest.unique())}')
print(f'problemas repetidos: {data.shape[0] - len(data.problem_statement.unique())}')

data.drop_duplicates(subset='problem_statement', inplace=True)

quantidade de problemas: 8343
quantidade de contests: 1424
problemas repetidos: 1524


## Pre-processamento

In [4]:
sw = set(stopwords.words('english'))
lemma = WordNetLemmatizer()
text = data.problem_statement

text = text.apply(lambda x: re.sub('[,|.|(|)|$|!|?|!]',' ',x)) # removendo caracteres especiais
text = text.apply(lambda x: [i for i in x.split() if i.lower() not in sw]) # removendo stop-words
text = text.apply(lambda x: ' '.join([lemma.lemmatize(i) for i in x])) # lemmatizando todo
# bigrama e trigrama todo

In [5]:
text

0       given n rectangle corner rectangle integer coo...
1       Daniel organizing football tournament come fol...
2       Piegirl found monster book monster pie reading...
3       far away land exists planet shaped like cylind...
4       Piegirl found red button one last chance chang...
                              ...                        
8338    n block arranged row numbered left right start...
8339    map capital Berland viewed infinite coordinate...
8340    play strategic video game yeah ran good proble...
8341    first let's define function f x follows: \begi...
8342    Recently lot student enrolled Berland State Un...
Name: problem_statement, Length: 6819, dtype: object

## Vertorizando

In [6]:
max_df = 0.8
min_df = 0.05
vec = CountVectorizer(max_df=max_df, min_df=min_df)

In [7]:
X = vec.fit_transform(text)
X.shape

(6819, 250)

## Agrupando

In [8]:
# Pegando quantidade de topicos

topics = data.problem_tags
total_topics = set()
topics = topics.apply(lambda x: re.sub('[*]*[0-9]','',x)) # numeros
topics = topics.apply(lambda x: [i for i in x.split(',') if i.lower() not in sw])
topics = topics.apply(lambda x: [total_topics.add(i) for i in x if i != ''])

In [9]:
n_total_topics = len(total_topics)
print(f'Número de Tópicos: {n_total_topics}')
total_topics

Número de Tópicos: 37


{'*specialproblem',
 '-sat',
 'binarysearch',
 'bitmasks',
 'bruteforce',
 'chineseremaindertheorem',
 'combinatorics',
 'constructivealgorithms',
 'datastructures',
 'dfsandsimilar',
 'divideandconquer',
 'dp',
 'dsu',
 'expressionparsing',
 'fft',
 'flows',
 'games',
 'geometry',
 'graphmatchings',
 'graphs',
 'greedy',
 'hashing',
 'implementation',
 'interactive',
 'math',
 'matrices',
 'meet-in-the-middle',
 'numbertheory',
 'probabilities',
 'schedules',
 'shortestpaths',
 'sortings',
 'strings',
 'stringsuffixstructures',
 'ternarysearch',
 'trees',
 'twopointers'}

In [10]:
n_components = n_total_topics
max_iter = 100

lda = LatentDirichletAllocation(n_components=n_components, max_iter=max_iter)

In [11]:
lda_matrix = lda.fit_transform(X)

In [12]:
top_word = lda.components_
terms = vec.get_feature_names_out()

for i, comp in enumerate(top_word):
    top_terms_key = zip(terms, comp)
    top_terms_key= sorted(top_terms_key, key = lambda t: t[1], reverse=True)[:10]
    top_terms_list= list(dict(top_terms_key).keys())
    print("Topic "+str(i)+": ",top_terms_list)

Topic 0:  ['game', 'play', 'turn', 'end', 'take', 'one', 'second', 'choose', 'beginning', 'initially']
Topic 1:  ['test', 'case', 'sum', 'exceed', 'one', 'answer', 'guaranteed', 'single', 'follow', 'description']
Topic 2:  ['set', 'problem', 'task', 'solve', 'solution', 'new', 'given', 'distinct', 'give', 'positive']
Topic 3:  ['print', 'without', 'yes', 'otherwise', 'quote', 'no', 'possible', 'impossible', 'two', 'whether']
Topic 4:  ['leq', '10', 'ldots', 'th', 'example', 'cdot', 'two', 'output', 'single', 'second']
Topic 5:  ['point', 'segment', 'coordinate', 'distance', 'two', 'xi', 'given', 'located', 'th', 'print']
Topic 6:  ['query', 'type', 'answer', 'two', 'one', 'th', 'next', 'second', 'following', 'given']
Topic 7:  ['space', 'separated', 'th', 'two', 'xi', 'people', 'single', 'next', '105', 'print']
Topic 8:  ['edge', 'vertex', 'graph', 'path', 'connected', 'given', 'two', 'contain', 'multiple', 'guaranteed']
Topic 9:  ['word', 'letter', 'lowercase', 'english', 'one', 'new'

## Validando

In [13]:
# todo

## Visualizando

In [14]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

panel = pyLDAvis.sklearn.prepare(lda, X, vec, mds='tsne')
panel
# todo

c:\Users\Bruno\Desktop\UNB\PIBIC\classificador-cf\venv\lib\site-packages\pyLDAvis\_prepare.py:244: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
28    -1.741718  2.370339       1        1  6.937414
22    -1.254553  2.165398       2        1  6.537149
31    -1.282526  2.138374       3        1  5.440511
7     -1.170653  2.325420       4        1  4.623213
1     -0.563121  2.848182       5        1  4.040932
24    -1.161814  3.083517       6        1  3.933630
23    -1.006839  2.518719       7        1  3.799039
15    -1.371031  1.768034       8        1  3.590134
18    -0.877587  2.385914       9        1  3.499214
32    -1.345960  1.944311      10        1  3.432840
26    -1.585955  2.441177      11        1  3.287685
3     -1.317822  2.331251      12        1  3.145186
5     -0.948543  2.613094      13        1  3.103191
27    -1.386779  2.615020      14        1  2.831862
17    -1.395959  2.499237      15        1  2.762068
30    -0.590704  2.064584      16        1  2.746970
36    -1.679089  2.410453      17        1  2.613713
4     -1.656794  2.393055      18        1  2.537249
8     -0.488904  2.533819      19        1  2.403839
6     -0.821779  2.279931      20        1  2.356463
35    -1.146233  3.292675      21        1  2.352190
20    -0.855701  2.104340      22        1  2.048191
13    -1.148610  1.608399      23        1  2.033118
14    -0.497732  2.576207      24        1  1.955061
11    -2.230388  2.452384      25        1  1.895617
16    -1.532851  3.293042      26        1  1.843610
21    -1.104648  1.711573      27        1  1.707859
0     -1.244553  1.482013      28        1  1.596486
12    -1.162542  3.478964      29        1  1.516625
10    -0.703948  2.252557      30        1  1.510923
2     -2.164684  2.974939      31        1  1.498409
9     -1.155970  2.844333      32        1  1.372482
34    -1.160190  3.663283      33        1  1.197388
33    -1.211651  1.784146      34        1  1.181290
29    -0.120288  1.672366      35        1  1.132713
19    -2.182662  1.628276      36        1  0.913073
25    -1.974957  3.338888      37        1  0.622663, topic_info=          Term          Freq         Total Category  logprob  loglift
125         le  13491.000000  13491.000000  Default  30.0000  30.0000
216     string   6978.000000   6978.000000  Default  29.0000  29.0000
220       test   6520.000000   6520.000000  Default  28.0000  28.0000
37        case   6394.000000   6394.000000  Default  27.0000  27.0000
200   sequence   4403.000000   4403.000000  Default  26.0000  26.0000
..         ...           ...           ...      ...      ...      ...
32   calculate      0.028282    709.077182  Topic37 -11.5323  -5.0506
77      either      0.028282   1085.765962  Topic37 -11.5323  -5.4767
0          000      0.028282   1410.434529  Topic37 -11.5323  -5.7383
1           10      0.028282   6512.943137  Topic37 -11.5323  -7.2682
3         1000      0.028282   1169.461170  Topic37 -11.5323  -5.5509

[1494 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
0         2  0.849384   000
0        15  0.043958   000
0        18  0.081535   000
0        24  0.024106   000
1         1  0.638575    10
...     ...       ...   ...
249      23  0.022925  zero
249      26  0.077651  zero
249      27  0.096139  zero
249      29  0.020707  zero
249      36  0.004437  zero

[2462 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[29, 23, 32, 8, 2, 25, 24, 16, 19, 33, 27, 4, 6, 28, 18, 31, 37, 5, 9, 7, 36, 21, 14, 15, 12, 17, 22, 1, 13, 11, 3, 10, 35, 34, 30, 20, 26])